In [ ]:
#  Read about Log. regression here 
#  https://en.wikipedia.org/wiki/Logistic_regression


In [1]:
import findspark
findspark.init()
import pyspark
from random import random
from pyspark import SparkContext

import sys
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.util import MLUtils

import numpy as np
import re

In [9]:
import os
port = 4040 + hash(os.getcwd().split("/")[2])%200
config = pyspark.SparkConf().set('spark.executor.memory', '512M')\
         .set('spark.cores.max', '2')\
         .set('spark.port.maxRetries','200')\
         .set('spark.ui.port', port)    
print port,     os.getcwd().split("/")[2]

4054 bkg318


In [10]:
sc = pyspark.SparkContext(appName="Logistic Regression", master='spark://polyp1:7077',conf = config )

In [11]:
data = sc.textFile("/scratch/ISE495/lab6_ML/ISLR.dat")

In [12]:
def parseData(line):
    data = line.strip().split(",")
    default = data[1]
    if default=='"No"':
        label=0
    else:
        label=1
        
    student = data[2]
    if student == '"No"':
        student =0
    else:
        student =1
    features=(float(data[3]),float(data[4]), student)  
    
    # you should fill this function
    
    return LabeledPoint(label,features)

parsedData = data.map(parseData)

# this will split randomly the data 60% for training and 40% for testing
trainingData, testData = parsedData.randomSplit( [0.6, 0.4] , seed = 11L)

In [13]:
# Find more inforamtion about parameteres at:
# https://spark.apache.org/docs/2.1.1/api/python/pyspark.mllib.html#pyspark.mllib.classification.LogisticRegressionWithLBFGS
model = LogisticRegressionWithLBFGS.train(trainingData, iterations=10, regParam=0.0001, regType='l2', intercept=True)

# Evaluating the model on training data
labelsAndPreds = trainingData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(trainingData.count())
print("Training Error = " + str(trainErr))


Training Error = 0.0350819129571


In [14]:
trainErr_test = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(testData.count())
#print("Testing Error = " + str(trainErr_test))

Accuracy=(1-trainErr_test)*100
print('Test Error = ' + str(trainErr_test))
print ('Accuracy = ' + str(Accuracy))

Test Error = 0.0535759413697
Accuracy = 94.642405863


In [ ]:
# we have used regParam=0.0001, try to use different values and plot training and testing 
# error as a function of regParam


In [ ]:
# now, create a copy of this jupyter notebook and run Log. Regression on the "SPAM" dataset

In [8]:
sc.stop()